# Machine interpreter

## Instructions

Four registers, `a`, `b`, `c`, and `d`.
Program counter, `pc`.

Each register can hold 8-byte integers (Java `long`s).

Machine carries out the instruction at location `pc`. After it's executed, `pc` increments by 1.

`jmp` and `jpz` override this normal change in `pc`.

| Instruction | Description |
|:------------|:------------|
| `inc r`     | increment contents of register `r` |
| `dec r`     | decrement contents of register `r` |
| `set r i`   | set contents of register `r` to literal value `i` |
| `cpy r s`   | copy contents of register `r` into register `s` | 
| `jmp i`     | jump to instruction `i` places forward |
| `jpz r i`   | jump to instruction `i` places forward if<br>register `r` contains zero, otherwise continue to next instruction |

For `jmp` and `jpz`, `i` is relative to the current instruction. `i` can be negative to jump to earlier places in the program. `i`=1 is a no-op, `i`=0 causes an infinite loop.

In [4]:
def new_machine():
    return {'pc': 0, 
            'a': 0,
            'b': 0, 
            'c': 0,
            'd': 0,
            'instructions': []}

In [5]:
def show_machine(machine):
    return ', '.join('{}: {}'.format(sk, machine[int(sk) if sk.isnumeric() else sk]) 
                     for sk in sorted(str(k) for k in machine)
                     if sk != 'instructions')

In [6]:
def inc(reg, machine):
    machine[reg] += 1
    machine['pc'] += 1

In [7]:
def dec(reg, machine):
    machine[reg] -= 1
    machine['pc'] += 1

In [8]:
def jmp(addr, machine):
    machine['pc'] += addr

In [9]:
def jpz(reg, addr, machine):
    if machine[reg] == 0:
        machine['pc'] += addr
    else:
        machine['pc'] += 1

In [10]:
def set_literal(reg, literal, machine):
    machine[reg] = literal
    machine['pc'] += 1

In [11]:
def cpy(from_reg, to_reg, machine):
    machine[to_reg] = machine[from_reg]
    machine['pc'] += 1

In [12]:
instruction_table = {'inc': inc, 'dec': dec, 'jmp': jmp,
                    'jpz': jpz, 'set': set_literal, 'cpy': cpy}
numeric_args_table = {'jmp': [0], 'jpz': [1], 'set': [1], 'sto': [1], 'ld': [1]}

In [13]:
def parse(instruction):
    words = instruction.split()
    instr = words[0]
    args = words[1:]
    if instr in numeric_args_table:
        for p in numeric_args_table[instr]:
            args[p] = int(args[p])
    return instruction_table[instr], args

In [14]:
m = new_machine()
inc('a', m)
cargs = ['a', 'b']
cpy(*cargs, m)
inc('a', m)
m

{'a': 2, 'b': 1, 'c': 0, 'd': 0, 'instructions': [], 'pc': 3}

In [15]:
def program_from_instructions(prog, machine):
    machine['instructions'] = [parse(instr) for instr in prog]

In [16]:
def program_from_listing(listing, machine):
    labelled_instructions = [i.strip() for i in listing.split('\n') 
                             if i.strip() 
                             if not i.strip().startswith('#')]
    instructions = replace_labels(labelled_instructions)
    program_from_instructions(instructions, machine)

In [17]:
def replace_labels(listing):
    locations = {}
    for n, i in enumerate(listing):
        if ':' in i:
            locations[i.split(':')[0]] = n

    unlabelled_listing = []
    for n, i in enumerate(listing):
        instr = i.split()
        if ':' in i:
            instr = i.split(':')[1].split()
        else:
            instr = i.split()
        terms = []
        for term in instr:
            if term in locations:
                terms += [str(locations[term] - n)]
            else:
                terms += [term]
        transformed_instr = ' '.join(terms)
        unlabelled_listing += [transformed_instr]
        
    return unlabelled_listing    

In [18]:
'fred: inc a'.split(':')[1].split()

['inc', 'a']

In [19]:
program = """
      set a 10
      # comment line
      
loop: dec a
      inc b
      jpz a 2
      jmp loop
"""
labelled_instructions = [i.strip() for i in program.split('\n') if i.strip() if not i.strip().startswith('#')]
instructions = replace_labels(labelled_instructions)
print('\n'.join(instructions))

set a 10
dec a
inc b
jpz a 2
jmp -3


In [20]:
def run(machine, initial_state=None, trace=False):
    if initial_state:
        machine.update(initial_state)
    while machine['pc'] < len(machine['instructions']):
        if trace:
            print(show_machine(machine))
        cmd, args = machine['instructions'][machine['pc']]
        cmd(*args, machine)

In [21]:
def execute(listing, initial_state=None, trace=False):
    m = new_machine()
    program_from_listing(listing, m)
    run(m, initial_state=initial_state, trace=trace)
    return m

In [22]:
program = """
inc a
inc a
cpy a b
inc a
"""
execute(program)

{'a': 3,
 'b': 2,
 'c': 0,
 'd': 0,
 'instructions': [(<function __main__.inc>, ['a']),
  (<function __main__.inc>, ['a']),
  (<function __main__.cpy>, ['a', 'b']),
  (<function __main__.inc>, ['a'])],
 'pc': 4}

In [24]:
program = """
set a 10
dec a
inc b
jpz a 2
jmp -3
"""
# m = new_machine()
# program_from_listing(program, m)
# run(m)
execute(program, initial_state={'c': 20})

{'a': 0,
 'b': 10,
 'c': 20,
 'd': 0,
 'instructions': [(<function __main__.set_literal>, ['a', 10]),
  (<function __main__.dec>, ['a']),
  (<function __main__.inc>, ['b']),
  (<function __main__.jpz>, ['a', 2]),
  (<function __main__.jmp>, [-3])],
 'pc': 5}

In [25]:
program = """
      set a 10
loop: dec a
      inc b
      jpz a 2
      jmp loop
"""
execute(program, initial_state={'c': 20})

{'a': 0,
 'b': 10,
 'c': 20,
 'd': 0,
 'instructions': [(<function __main__.set_literal>, ['a', 10]),
  (<function __main__.dec>, ['a']),
  (<function __main__.inc>, ['b']),
  (<function __main__.jpz>, ['a', 2]),
  (<function __main__.jmp>, [-3])],
 'pc': 5}

In [26]:
program = """
cpy c a
set b 0
dec a
jpz b 3
dec b
jmp 2
inc b
jpz a 3
jmp -6
"""
# m = new_machine()
# program_from_listing(program, m)
# run(m)
execute(program, initial_state={'c': 5})

{'a': 0,
 'b': 1,
 'c': 5,
 'd': 0,
 'instructions': [(<function __main__.cpy>, ['c', 'a']),
  (<function __main__.set_literal>, ['b', 0]),
  (<function __main__.dec>, ['a']),
  (<function __main__.jpz>, ['b', 3]),
  (<function __main__.dec>, ['b']),
  (<function __main__.jmp>, [2]),
  (<function __main__.inc>, ['b']),
  (<function __main__.jpz>, ['a', 3]),
  (<function __main__.jmp>, [-6])],
 'pc': 10}

In [27]:
# b holds parity of number in c: (c % 2)
program = """
      cpy c a
      set b 0
loop: dec a
      jpz b odd
      dec b
      jmp end
odd:  inc b
end:  jpz a 2
      jmp loop
"""
# m = new_machine()
# program_from_listing(program, m)
# run(m)
execute(program, initial_state={'c': 5})

{'a': 0,
 'b': 1,
 'c': 5,
 'd': 0,
 'instructions': [(<function __main__.cpy>, ['c', 'a']),
  (<function __main__.set_literal>, ['b', 0]),
  (<function __main__.dec>, ['a']),
  (<function __main__.jpz>, ['b', 3]),
  (<function __main__.dec>, ['b']),
  (<function __main__.jmp>, [2]),
  (<function __main__.inc>, ['b']),
  (<function __main__.jpz>, ['a', 2]),
  (<function __main__.jmp>, [-6])],
 'pc': 9}

In [28]:
# c holds floor(a/2)
program = """
      set c 0
      set b 0
loop: dec a
      jpz b odd
      dec b
      inc c
      jmp end
odd:  inc b
end:  jpz a 2
      jmp loop
"""
# m = new_machine()
# program_from_listing(program, m)
# run(m)
show_machine(execute(program, initial_state={'a': 17}))

'a: 0, b: 1, c: 8, d: 0, pc: 10'

In [29]:
# c holds a * 3
program = """
      set c 0
      cpy a b
      # start of main loop
loop: jpz b end
      dec b
      inc c
      inc c
      inc c
      jmp loop
      
      # end of program 
      
end:  jmp 1
"""
# m = new_machine()
# program_from_listing(program, m)
# run(m)
show_machine(execute(program, initial_state={'a': 4}))

'a: 4, b: 0, c: 12, d: 0, pc: 9'

In [38]:
# c holds a * b
program = """
      set c 0
      cpy a d
loop: jpz b end      
      dec b
      cpy d a
smul: jpz a emul
      inc c
      dec a
      jmp smul
emul: jmp loop 
      
end:  set d 0
"""
m = new_machine()
program_from_listing(program, m)
run(m)
show_machine(execute(program, initial_state={'a': 9, 'b': 3}))

'a: 0, b: 0, c: 27, d: 0, pc: 11'

In [39]:
labelled_instructions = [i.strip() for i in program.split('\n') if i.strip() if not i.strip().startswith('#')]
instructions = replace_labels(labelled_instructions)
print('\n'.join(instructions))

set c 0
cpy a d
jpz b 8
dec b
cpy d a
jpz a 4
inc c
dec a
jmp -3
jmp -7
set d 0


In [31]:
# c holds a * b
program = """
      set a 2
      set b 5
      set c 0
      cpy a d
loop: jpz b end      
      dec b
      cpy d a
smul: jpz a emul
      inc c
      dec a
      jmp smul
emul: jmp loop 
      
end:  cpy d a
"""
m = new_machine()
program_from_listing(program, m)
run(m)
show_machine(execute(program, initial_state={'a': 9, 'b': 3}))

'a: 2, b: 0, c: 10, d: 2, pc: 13'

In [32]:
labelled_instructions = [i.strip() for i in program.split('\n') 
                             if i.strip() 
                             if not i.strip().startswith('#')]
instructions = replace_labels(labelled_instructions)
print('\n'.join(instructions))

set a 2
set b 5
set c 0
cpy a d
jpz b 8
dec b
cpy d a
jpz a 4
inc c
dec a
jmp -3
jmp -7
cpy d a


In [33]:
# Collatz. a initially holds value, but location 1 is used to store the current value as we're going along.
# Location 2 holds number of steps taken
# Location 3 holds max value reached
program = """
      cpy a d
      
      set b 0
      
      # if a is one, finish, 
main: dec a
      jpz a end
      inc a
      
      # find parity of a
      cpy a c
      set b 0
prty: dec c
      jpz b odd
      dec b
      jmp prte
odd:  inc b
prte: jpz c 2
      jmp prty
      
      # b == 0 means a even; b == 1 means a odd
      jpz b isev

      # c holds a * 3 + 1
      cpy a b
mul:  jpz b emul
      dec b
      inc c
      inc c
      inc c
      jmp mul
emul: inc c
      cpy c a
      jmp fin
      
      
isev: set c 0
      set b 0
hlvl: dec a
      jpz b oddh
      dec b
      inc c
      jmp endh
oddh: inc b
endh: jpz a 2
      jmp hlvl
      cpy c a

fin:  cpy a b
      cpy d c
maxc: jpz c this
      jpz b othr
      dec b
      dec c
      jmp maxc
this: cpy a d
othr: jmp main
      
      # end of program 
      
end:  cpy d a
      set c 0
      set d 0
"""
# m = new_machine()
# program_from_listing(program, m)
# run(m)
show_machine(execute(program, initial_state={'a': 7}))

'a: 52, b: 0, c: 0, d: 0, pc: 48'

In [32]:
13*3+1

40

In [33]:
def max_collatz(start):
    mc = start
    i = start
    while i != 1:
        if i % 2 == 0:
            i = i // 2
        else:
            i = 3 * i + 1
        if i > mc:
            mc = i
    return mc

In [34]:
max_collatz(7)

52

In [35]:
max([(max_collatz(i), i) for i in range(1, 1000)])

(250504, 937)

In [36]:
ans = []
for i in range(1, 100):
    m = execute(program, initial_state={'a': i})
    c = max_collatz(i)
    if m[1] != c:
        ans += [(i, m[1], c)]
ans

[(1, 0, 1),
 (2, 0, 2),
 (3, 0, 16),
 (4, 0, 4),
 (5, 0, 16),
 (6, 0, 16),
 (7, 0, 52),
 (8, 0, 8),
 (9, 0, 52),
 (10, 0, 16),
 (11, 0, 52),
 (12, 0, 16),
 (13, 0, 40),
 (14, 0, 52),
 (15, 0, 160),
 (16, 0, 16),
 (17, 0, 52),
 (18, 0, 52),
 (19, 0, 88),
 (20, 0, 20),
 (21, 0, 64),
 (22, 0, 52),
 (23, 0, 160),
 (24, 0, 24),
 (25, 0, 88),
 (26, 0, 40),
 (27, 0, 9232),
 (28, 0, 52),
 (29, 0, 88),
 (30, 0, 160),
 (31, 0, 9232),
 (32, 0, 32),
 (33, 0, 100),
 (34, 0, 52),
 (35, 0, 160),
 (36, 0, 52),
 (37, 0, 112),
 (38, 0, 88),
 (39, 0, 304),
 (40, 0, 40),
 (41, 0, 9232),
 (42, 0, 64),
 (43, 0, 196),
 (44, 0, 52),
 (45, 0, 136),
 (46, 0, 160),
 (47, 0, 9232),
 (48, 0, 48),
 (49, 0, 148),
 (50, 0, 88),
 (51, 0, 232),
 (52, 0, 52),
 (53, 0, 160),
 (54, 0, 9232),
 (55, 0, 9232),
 (56, 0, 56),
 (57, 0, 196),
 (58, 0, 88),
 (59, 0, 304),
 (60, 0, 160),
 (61, 0, 184),
 (62, 0, 9232),
 (63, 0, 9232),
 (64, 0, 64),
 (65, 0, 196),
 (66, 0, 100),
 (67, 0, 304),
 (68, 0, 68),
 (69, 0, 208),
 (70, 0, 1

In [34]:
show_machine(execute(program, initial_state={'a': 937}))

'a: 250504, b: 0, c: 0, d: 0, pc: 48'

In [35]:
labelled_instructions = [i.strip() for i in program.split('\n') 
                             if i.strip() 
                             if not i.strip().startswith('#')]
instructions = replace_labels(labelled_instructions)
print('\n'.join(instructions))
open('07-program.txt', 'w').write('\n'.join(instructions))

cpy a d
set b 0
dec a
jpz a 42
inc a
cpy a c
set b 0
dec c
jpz b 3
dec b
jmp 2
inc b
jpz c 2
jmp -6
jpz b 11
cpy a b
jpz b 6
dec b
inc c
inc c
inc c
jmp -5
inc c
cpy c a
jmp 12
set c 0
set b 0
dec a
jpz b 4
dec b
inc c
jmp 2
inc b
jpz a 2
jmp -7
cpy c a
cpy a b
cpy d c
jpz c 5
jpz b 5
dec b
dec c
jmp -4
cpy a d
jmp -42
cpy d a
set c 0
set d 0


344